Import the necessary libraries and dependencies

In [1]:
from python_lib.BasicModel import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *
import pandas as pd


# Model initialisation 

Use BlockSave to save all the weights into a `.bin` file

Be sure to `load_state_dict` from a `.pt`/`.pth` file before BlockSave

It's omitted here as I don't have the weights on hand, so I just use a random initialised weights.

In [2]:
model = BasicModel(input_size = 2, input_length=16).to("cuda")

BlockSave(model.return_layers(), "fullbasicmodel", "verifyCppVsPy")


# Generate Input Feats for testing purposes

In [4]:
input_feats = torch.rand([1,2,16]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,16))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("BasicModelWeights", f"basicinput_2x16.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.28700072 0.6169604  0.6852525  0.57610816 0.95842755 0.69390893
  0.09680605 0.83381253 0.89192003 0.8134774  0.9693227  0.14813513
  0.27104497 0.64710045 0.7205885  0.49191618]
 [0.6322167  0.04847664 0.34838754 0.17704517 0.09914815 0.8611054
  0.7872366  0.30172074 0.4570409  0.93936247 0.26015586 0.19969726
  0.3091157  0.33220154 0.22918075 0.94229597]]


# Run the model using eval/train mode

In [44]:
model.eval()
print(input_feats)
model(input_feats)

tensor([[[0.2870, 0.6170, 0.6853, 0.5761, 0.9584, 0.6939, 0.0968, 0.8338,
          0.8919, 0.8135, 0.9693, 0.1481, 0.2710, 0.6471, 0.7206, 0.4919],
         [0.6322, 0.0485, 0.3484, 0.1770, 0.0991, 0.8611, 0.7872, 0.3017,
          0.4570, 0.9394, 0.2602, 0.1997, 0.3091, 0.3322, 0.2292, 0.9423]]],
       device='cuda:0')


tensor([[0.1855, 0.1936, 0.1554, 0.1603, 0.1642, 0.1411]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

# Verification Test C++ vs Py

## Generating Test Inputs

**NOTE:** Change the values of `no_of_inputs, no_of_channel_in, no_of_input_width` accordingly to desired matrix size

Randomizes a `no_of_inputs x no_of_channel_in x no_of_input_width` matrix and saves the values into:
1. A binary (to feed C++ as input)
2. A CSV (to verify input values)

In [3]:
no_of_inputs = 10
no_of_channel_in = 2
no_of_input_width = 16

verify_feats = torch.rand([no_of_inputs,no_of_channel_in,no_of_input_width]).to("cuda")
print(verify_feats)
verify_feats_np = verify_feats.cpu().detach().numpy()

## Reshaping
# if no_of_inputs == 1:
#     verify_feats_np = np.reshape(verify_feats_np, (no_of_channel_in,no_of_input_width))
# else:
verify_feats_np = np.reshape(verify_feats_np, (no_of_inputs,no_of_channel_in,no_of_input_width)) 
dim = verify_feats_np.shape
print(dim)

## Writing verify inputs into binary
flatten_verify = verify_feats_np.flatten()
with open(os.path.join("verifyCppVsPy", f"testInput_basicModel_{no_of_inputs}x{no_of_channel_in}x{no_of_input_width}.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_verify.tobytes())
            
## Extracting verification input values into CSV
array_2d = verify_feats_np.reshape(no_of_inputs, -1)
output_df = pd.DataFrame(array_2d)
output_df.to_csv(f"verifyCppVsPy/testInput_basicModel_{no_of_inputs}x{no_of_channel_in}x{no_of_input_width}.csv",\
                 header=False, index=False)
print(verify_feats_np)

tensor([[[0.8133, 0.8277, 0.3606, 0.5747, 0.1968, 0.5832, 0.6849, 0.6439,
          0.4271, 0.5965, 0.1522, 0.4538, 0.5138, 0.9584, 0.2000, 0.4415],
         [0.3546, 0.0429, 0.7636, 0.9730, 0.3013, 0.7972, 0.4903, 0.7709,
          0.7257, 0.0981, 0.4050, 0.5893, 0.1055, 0.9590, 0.1994, 0.3371]],

        [[0.4814, 0.3528, 0.2498, 0.3486, 0.0058, 0.7663, 0.5095, 0.1951,
          0.9229, 0.9356, 0.8047, 0.3925, 0.3932, 0.8909, 0.5320, 0.8430],
         [0.0116, 0.5949, 0.5425, 0.7190, 0.4351, 0.5252, 0.4236, 0.4051,
          0.5414, 0.8874, 0.2214, 0.2265, 0.8808, 0.8040, 0.2536, 0.2125]],

        [[0.2000, 0.6315, 0.1979, 0.2110, 0.4822, 0.0650, 0.4383, 0.7137,
          0.0827, 0.1014, 0.1613, 0.4584, 0.8579, 0.8029, 0.3671, 0.4466],
         [0.9447, 0.7537, 0.6527, 0.0850, 0.8064, 0.6009, 0.6012, 0.0357,
          0.2287, 0.3579, 0.7994, 0.2310, 0.0509, 0.8649, 0.1967, 0.5335]],

        [[0.5226, 0.9533, 0.0495, 0.0578, 0.1125, 0.8175, 0.2240, 0.0174,
          0.6366, 0.0358, 

In [4]:
model.eval()
model(verify_feats)

/home/tyouqun/Git/LegacyCNN.wt/verificationTest/python_lib/BasicModel.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = F.softmax(y)


tensor([[0.1475, 0.2188, 0.1529, 0.1597, 0.1640, 0.1571],
        [0.1475, 0.2189, 0.1529, 0.1596, 0.1641, 0.1571],
        [0.1474, 0.2191, 0.1529, 0.1595, 0.1640, 0.1571],
        [0.1475, 0.2187, 0.1528, 0.1597, 0.1641, 0.1571],
        [0.1475, 0.2190, 0.1529, 0.1595, 0.1640, 0.1571],
        [0.1475, 0.2191, 0.1530, 0.1596, 0.1638, 0.1570],
        [0.1474, 0.2190, 0.1528, 0.1595, 0.1641, 0.1571],
        [0.1475, 0.2189, 0.1529, 0.1596, 0.1640, 0.1571],
        [0.1475, 0.2189, 0.1528, 0.1596, 0.1642, 0.1571],
        [0.1474, 0.2192, 0.1530, 0.1595, 0.1639, 0.1570]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [5]:
model(verify_feats).cpu().detach().numpy()

array([[0.14749992, 0.2187969 , 0.15289658, 0.15965511, 0.16404627,
        0.15710515],
       [0.1474942 , 0.21893455, 0.1528505 , 0.15956768, 0.16407992,
        0.15707314],
       [0.14742531, 0.21911602, 0.15294908, 0.15949346, 0.16396283,
        0.15705332],
       [0.14753692, 0.21870603, 0.1528481 , 0.15966213, 0.164131  ,
        0.1571158 ],
       [0.14752172, 0.21895556, 0.15287286, 0.15954831, 0.16404958,
        0.157052  ],
       [0.14745808, 0.21914601, 0.15300055, 0.15958494, 0.16381305,
        0.15699741],
       [0.14744994, 0.21897846, 0.15284084, 0.15954459, 0.16409242,
        0.15709376],
       [0.1475111 , 0.21891044, 0.15289554, 0.15961672, 0.16400811,
        0.15705808],
       [0.14747508, 0.21885875, 0.15275142, 0.15956622, 0.16419949,
        0.15714899],
       [0.14742728, 0.21919176, 0.15297556, 0.15951811, 0.16387963,
        0.15700766]], dtype=float32)